# Final Project - Reinforcements Learning 
Hello dear students,<br> this is the template notebook. Please click on the "File" tab and then on "Save a copy into drive".

---
<br>

### Name and ID:
Student 1: Avraham Raviv, 204355390
<br>
Student 2: Yevgeni Berkovitch, 317079234
<br><br>
<img src="https://play-lh.googleusercontent.com/e_oKlKPISbgdzut1H9opevS7-LTB8-8lsmpCdMkhlnqFenZhpjxbLmx7l158-xQQCIY">

### https://github.com/mpSchrader/gym-sokoban

# Installs

In [1]:
%%capture
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install 'imageio==2.4.0'
!pip install gym
!pip install pygame
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install pyvirtualdisplay
!pip install piglet
!pip install gym
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install gym_sokoban

!imageio_download_bin ffmpeg

# Imports

In [2]:
import random
import time

import numpy as np
import scipy as scp
import matplotlib.pyplot as plt

import base64
import imageio
from pyvirtualdisplay import Display
from IPython.display import HTML

import gym
from gym import error, spaces, utils
from soko_pap import *

from collections import deque
from queue import PriorityQueue

from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten

from tqdm.notebook import tqdm
from collections import defaultdict

In [3]:
%matplotlib inline

In [4]:
imageio.plugins.ffmpeg.download()

In [5]:
from gym import logger as gymlogger
gymlogger.set_level(40) # error only

# Display utils
The cell below contains the video display configuration. No need to make changes here.

In [6]:
def embed_mp4(filename):
    """Embeds an mp4 file in the notebook."""
    video = open(filename,'rb').read()
    b64 = base64.b64encode(video)
    tag = '''
    <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
    Your browser does not support the video tag.
    </video>'''.format(b64.decode())

    return HTML(tag)

# Utils

In [7]:
def get_distances(room_state):
    for i in range(room_state.shape[0]):
        for j in range(room_state.shape[1]):
            if room_state[i][j] == 2:
                target = (i, j)

    distances = np.zeros(shape=room_state.shape)
    visited_cells = set()
    cell_queue = deque()

    visited_cells.add(target)
    cell_queue.appendleft(target)

    while len(cell_queue) != 0:
        cell = cell_queue.pop()
        distance = distances[cell[0]][cell[1]]
        for x,y in ((1,0), (-1,-0), (0,1), (0,-1)):
            next_cell_x, next_cell_y = cell[0]+x, cell[1]+y
            if room_state[next_cell_x][next_cell_y] != 0 and not (next_cell_x, next_cell_y) in visited_cells:
                distances[next_cell_x][next_cell_y] = distance + 1
                visited_cells.add((next_cell_x, next_cell_y))
                cell_queue.appendleft((next_cell_x, next_cell_y))
                
    return distances   

def calc_distances(room_state, distances):
    box = None
    mover = None
    for i in range(room_state.shape[0]):
        for j in range(room_state.shape[1]):            
            if room_state[i][j] == 4:
                box = (i,j)
            
            if room_state[i][j] == 5:
                mover = (i,j)
    
    return mover, box, distances[box[0]][box[1]]   

def box2target_change_reward(room_state, next_room_state, distances):
    if np.array_equal(room_state, next_room_state):
        return -1.0
    
    mover, box, t2b = calc_distances(room_state, distances)
    n_mover, n_box, n_t2b = calc_distances(next_room_state, distances)
    
    change_reward = 0.0
    if n_t2b < t2b:
        change_reward += 5.0
    elif n_t2b > t2b:
        change_reward -= 5.0
        
    m2b = np.sqrt((mover[0]-box[0])**2 + (mover[1]-box[1])**2)
    n_m2b = np.sqrt((n_mover[0]-n_box[0])**2 + (n_mover[1]-n_box[1])**2)
    
    if n_m2b < m2b and m2b >= 2:
        change_reward += 1.0
    elif n_m2b > m2b and n_m2b >= 2:
        change_reward -= 1.0
        
    return change_reward   

# Solution

In [8]:
class SOK_Agent:
    def __init__(self):
        # Construct DQN models
        self.state_size = (5, 5, 1)  
        self.action_size = 8
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.target_model.set_weights(self.model.get_weights())
        self.batch_size = 8
        
        # Replay buffers
        self.replay_buffer = deque(maxlen=5000)
        self.prioritized_replay_buffer = deque(maxlen=500)
        self.prioritized_replay_batch = 50        
        
        # Hyperparameters
        self.gamma = 0.9
        self.epsilon = 1.0   
        self.epsilon_min = 0.3
        self.epsilon_decay = 0.995
        self.replay_rate = 10
        self.update_beta = 0.999

        self.verbosity = 100 

    def _build_model(self):
        model = Sequential()
        model.add(Conv2D(32, (3, 3), input_shape=self.state_size, activation='relu'))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(Flatten())
        model.add(Dense(32, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer="adam")        
        return model

    def remember(self, state, action, reward, next_state, done):
        self.replay_buffer.append([state, action, reward, next_state, done])    
        
    def copy_to_prioritized_buffer(self, n):
        for i in range(n):
            self.prioritized_replay_buffer.append(self.replay_buffer[-1-i])  

    def act(self, state, stochastic=False):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        
        act_values = self.model.predict(state, verbose=0)[0]
        
        if stochastic:
            act_probs = np.exp(act_values)/np.exp(act_values).sum()
            return np.random.choice(np.arange(self.action_size), size=1, p=act_probs)[0]
              
        return np.argmax(act_values) 

    def replay(self): 
        if len(self.replay_buffer) < self.batch_size:
            return
        
        if len(self.prioritized_replay_buffer) < self.batch_size//2:
            minibatch = random.sample(self.replay_buffer, self.batch_size) 
        else:    
            minibatch = random.sample(self.replay_buffer, self.batch_size//2) 
            minibatch.extend(random.sample(self.prioritized_replay_buffer, self.batch_size//2))
        
        states = np.zeros((self.batch_size, self.state_size[0], self.state_size[1]))
        actions = np.zeros(self.batch_size, dtype=int)
        rewards = np.zeros(self.batch_size)
        next_states = np.zeros((self.batch_size, self.state_size[0], self.state_size[1]))
        statuses = np.zeros(self.batch_size)
        targets = np.zeros((self.batch_size, self.action_size)) 
        
        for i, (state, action, reward, next_state, done) in enumerate(minibatch): 
            states[i] = state.copy()
            actions[i] = action
            rewards[i] = reward
            next_states[i] = next_state.copy()
            statuses[i] = 1 if done else 0    
        
        targets = self.model.predict(states) 
        max_actions = np.argmax(self.model.predict(next_states), axis=1)
        next_rewards = self.target_model.predict(next_states)
        
        ind = 0
        for action, reward, next_reward, max_action, done in zip(actions, rewards, next_rewards, max_actions, statuses):  
            if not done:
                reward += self.gamma * next_reward[max_action]
            targets[ind][action] = reward
            ind += 1
        
        self.model.fit(states, targets, epochs=10, verbose=0) 
        
        self.update_target_model()        
    
        if self.epsilon > self.epsilon_min:
            self.epsilon = self.epsilon * self.epsilon_decay
        
    def update_target_model(self):
        model_w = self.model.get_weights()
        target_model_w = self.target_model.get_weights()
        updated_target_model_w = []
        for i in range(len(model_w)):
            updated_target_model_w.append(self.update_beta*target_model_w[i] + (1-self.update_beta)*model_w[i])
        self.target_model.set_weights(updated_target_model_w)    
            
    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [9]:
def process_frame(frame):
    f = frame[16:96, 16:96, 0]   
    f = f.reshape(5, 16, 5, 16).max(axis=(1, 3))
    #f = f.flatten()
    f = f / 255
    return np.expand_dims(f, axis=0)

## Training

#### Test Suite

In [10]:
def test_agent(stochastic=False):
    current_epsilon = agent.epsilon
    agent.epsilon = 0.0
    num_solved = 0
    solved_in_steps = defaultdict(int)

    for t in tqdm(range(100)):    
        random.seed(t)
        sok = PushAndPullSokobanEnv(dim_room=(7, 7), num_boxes=1)
        sok.set_maxsteps(20)
        steps = 0

        state = sok.get_image('rgb_array')
        done = False
        while not done:
            steps += 1
            action = agent.act(process_frame(state), stochastic)
            if action < 4:
                action += 1
            else:
                action += 5
            state, reward, done, info = sok.step(action)

        if 3 in sok.room_state:            
            num_solved += 1
            solved_in_steps[steps] += 1
    
    agent.epsilon = current_epsilon
    print("*" * 30)
    print("Stochastic" if stochastic else "Deterministic")
    print("*" * 30)
    print("Solved: %d" % num_solved)
    print("=" * 30)
    print(solved_in_steps)
    print("*" * 30)

In [11]:
max_episodes = 5000
max_steps = 100

def init_sok(r):
    random.seed(r+100)
    sok = PushAndPullSokobanEnv(dim_room=(7, 7), num_boxes=1)
    sok.set_maxsteps(max_steps)
    return sok

In [ ]:
agent = SOK_Agent()

steps_per_episode = []

for e in range(max_episodes):
    if e % 100 == 0:
        test_agent(stochastic=False)
        test_agent(stochastic=True)
        
    print("Episode: %d" % (e))
    
    sok = init_sok(e)
    random.seed(e)
    
    state = process_frame(sok.get_image('rgb_array'))
    room_state = sok.room_state.copy() 
    distances = get_distances(room_state)
    
    for step in range(sok.max_steps):
        action = agent.act(state)
        if action < 4:
            next_state, reward, done, _ = sok.step(action+1) 
        else:
            next_state, reward, done, _ = sok.step(action+5)         
        
        next_state = process_frame(next_state)        
        next_room_state = sok.room_state
        
        if not done:
            reward += box2target_change_reward(room_state, next_room_state, distances)
        
        agent.remember(state, action, reward, next_state, done)
        
        state = next_state.copy() 
        room_state = next_room_state.copy()                
        
        if (step+1) % agent.replay_rate == 0:
            agent.replay()            
        
        if done:   
            steps_per_episode.append(step+1)
            
            if 3 in sok.room_state:                
                print("SOLVED! Episode %d Steps: %d Epsilon %.4f" % (e, step+1, agent.epsilon)) 
                agent.copy_to_prioritized_buffer(min(agent.prioritized_replay_batch, step+1))
            
            #agent.save("exp1_episode%d.h5" % (e))            
            break


******************************
Deterministic
******************************
Solved: 5
defaultdict(<class 'int'>, {2: 2, 1: 1, 3: 2})
******************************



******************************
Stochastic
******************************
Solved: 12
defaultdict(<class 'int'>, {15: 1, 6: 4, 3: 3, 14: 2, 4: 1, 8: 1})
******************************
Episode: 0
Episode: 1
Episode: 2
Episode: 3
Episode: 4
SOLVED! Episode 4 Steps: 95 Epsilon 0.7822
Episode: 5
Episode: 6
Episode: 7
SOLVED! Episode 7 Steps: 9 Epsilon 0.7076
Episode: 8
Episode: 9
Episode: 10
Episode: 11
Episode: 12
Episode: 13
Episode: 14
SOLVED! Episode 14 Steps: 78 Epsilon 0.5058
Episode: 15
Episode: 16
Episode: 17
Episode: 18
Episode: 19
SOLVED! Episode 19 Steps: 3 Epsilon 0.4139
Episode: 20
Episode: 21
Episode: 22
SOLVED! Episode 22 Steps: 1 Epsilon 0.3744
Episode: 23
Episode: 24
SOLVED! Episode 24 Steps: 35 Epsilon 0.3508
Episode: 25
Episode: 26
Episode: 27
Episode: 28
SOLVED! Episode 28 Steps: 3 Epsilon 0.3018
Episode: 29
Episode: 30
Episode: 31
Episode: 32
Episode: 33
SOLVED! Episode 33 Steps: 100 Epsilon 0.2988
Episode: 34
Episode: 35
Episode: 36
Episode: 37
Episode: 38
Episode: 39



******************************
Deterministic
******************************
Solved: 12
defaultdict(<class 'int'>, {1: 8, 2: 2, 3: 2})
******************************



******************************
Stochastic
******************************
Solved: 17
defaultdict(<class 'int'>, {6: 4, 2: 1, 4: 3, 3: 2, 11: 1, 1: 3, 13: 1, 9: 1, 16: 1})
******************************
Episode: 100
Episode: 101
Episode: 102
Episode: 103
Episode: 104
Episode: 105
Episode: 106
Episode: 107
Episode: 108
Episode: 109
Episode: 110
Episode: 111
Episode: 112
Episode: 113
SOLVED! Episode 113 Steps: 1 Epsilon 0.2988
Episode: 114
SOLVED! Episode 114 Steps: 1 Epsilon 0.2988
Episode: 115
Episode: 116
SOLVED! Episode 116 Steps: 3 Epsilon 0.2988
Episode: 117
Episode: 118
Episode: 119
Episode: 120
Episode: 121
Episode: 122
Episode: 123
SOLVED! Episode 123 Steps: 3 Epsilon 0.2988
Episode: 124
Episode: 125
SOLVED! Episode 125 Steps: 2 Epsilon 0.2988
Episode: 126
Episode: 127
SOLVED! Episode 127 Steps: 45 Epsilon 0.2988
Episode: 128
Episode: 129
SOLVED! Episode 129 Steps: 45 Epsilon 0.2988
Episode: 130
Episode: 131
Episode: 132
Episode: 133
SOLVED! Episode 133 Steps: 3 Epsilon 0.2988
Ep


******************************
Deterministic
******************************
Solved: 24
defaultdict(<class 'int'>, {3: 9, 2: 5, 1: 10})
******************************


...


******************************
Deterministic
******************************
Solved: 27
defaultdict(<class 'int'>, {3: 12, 2: 6, 1: 9})
******************************



******************************
Stochastic
******************************
Solved: 34
defaultdict(<class 'int'>, {3: 13, 7: 3, 2: 7, 1: 6, 9: 1, 12: 1, 20: 2, 4: 1})
******************************
Episode: 1600
SOLVED! Episode 1600 Steps: 2 Epsilon 0.2988
Episode: 1601
SOLVED! Episode 1601 Steps: 18 Epsilon 0.2988
Episode: 1602
SOLVED! Episode 1602 Steps: 74 Epsilon 0.2988
Episode: 1603
SOLVED! Episode 1603 Steps: 3 Epsilon 0.2988
Episode: 1604
Episode: 1605
SOLVED! Episode 1605 Steps: 29 Epsilon 0.2988
Episode: 1606
Episode: 1607
Episode: 1608
Episode: 1609
Episode: 1610
Episode: 1611
Episode: 1612
SOLVED! Episode 1612 Steps: 13 Epsilon 0.2988
Episode: 1613
Episode: 1614
Episode: 1615
SOLVED! Episode 1615 Steps: 2 Epsilon 0.2988
Episode: 1616
Episode: 1617
Episode: 1618
SOLVED! Episode 1618 Steps: 3 Epsilon 0.2988
Episode: 1619
SOLVED! Episode 1619 Steps: 6 Epsilon 0.2988
Episode: 1620
Episode: 1621
Episode: 1622
SOLVED! Episode 1622 Steps: 1 Epsilon 0.2988
Episode: 1623
SOLVED! Episod


******************************
Deterministic
******************************
Solved: 33
defaultdict(<class 'int'>, {3: 16, 2: 7, 1: 10})
******************************



******************************
Stochastic
******************************
Solved: 37
defaultdict(<class 'int'>, {3: 16, 4: 2, 7: 1, 2: 6, 1: 9, 5: 2, 16: 1})
******************************
Episode: 1700
Episode: 1701
Episode: 1702
Episode: 1703
Episode: 1704
SOLVED! Episode 1704 Steps: 1 Epsilon 0.2988
Episode: 1705
Episode: 1706
SOLVED! Episode 1706 Steps: 36 Epsilon 0.2988
Episode: 1707
SOLVED! Episode 1707 Steps: 63 Epsilon 0.2988
Episode: 1708
Episode: 1709
SOLVED! Episode 1709 Steps: 3 Epsilon 0.2988
Episode: 1710
Episode: 1711
Episode: 1712
Episode: 1713
Episode: 1714
SOLVED! Episode 1714 Steps: 33 Epsilon 0.2988
Episode: 1715
Episode: 1716
SOLVED! Episode 1716 Steps: 92 Epsilon 0.2988
Episode: 1717
Episode: 1718
SOLVED! Episode 1718 Steps: 8 Epsilon 0.2988
Episode: 1719
Episode: 1720
SOLVED! Episode 1720 Steps: 2 Epsilon 0.2988
Episode: 1721
Episode: 1722
SOLVED! Episode 1722 Steps: 3 Epsilon 0.2988
Episode: 1723
SOLVED! Episode 1723 Steps: 1 Epsilon 0.2988
Episode: 1724
Episode


******************************
Deterministic
******************************
Solved: 26
defaultdict(<class 'int'>, {3: 11, 1: 9, 2: 6})
******************************
